In [5]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import linregress
from IPython.core.display import HTML
import sklearn
from statsmodels.graphics.factorplots import interaction_plot
from statsmodels.stats.anova import anova_lm, AnovaRM
import pingouin as pg

from utility_functions import *

In [53]:
# Constants and Variables
DATA_PATH = './../data/cleaned/'

# Read data from files
df_full = pd.read_csv(f'{DATA_PATH}main-20-12-14-processed_filtered.csv')

# df_full=correct_condition(df_full)

/var/folders/xf/299x0l456pl3nxbc1jbwpyfc0000gn/T/ipykernel_1527/3899779587.py:5: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv(f'{DATA_PATH}main-20-12-14-processed_filtered.csv')


In [95]:
# Exclude participants
df_full = df_full[~df_full['pid'].isin(['5f10da001b822c2988fbdb89', '5fbe9e4810850d05920f1d58', '5fbd4036ab05790a26289296'])]
# ZERO CONFIDENCE RATING
print(df_trials[(df_trials['confidence']==0)][['pid', 'confidence']].groupby('pid').count()) #['pid'].unique()unts)


                          confidence
pid                                 
5aeb3c327ed5460001572a5e           1
5cd951d48ace520016ef7a31           1
5d100e740277ff00152f7562          10
5d7a8c6e553a7b001a32d9fd           3
5ddc03948db4ed000c4bb499           1
5de325f6df86d334c8894e69           2
5e70bd5480f43a0009625d4c           1
5e99cf5b6b2d720364b92162           1
5eb8ff91a1bdb386c80cd7ea           3
5ec47b2d91b37736ce88f3d6           3
5ed403a89542210ab77530b7           1
5ee152b0950c3853a265eab2           1
5f2075147b2ccb1408401905          13
5f25419d6c25411a2b68ddd8           1
5f5d3cc76943f05807b31df5          23
5f615839b063530fd7e1fffc           2
5f97db802b5a61000ad750ec           1
5fab66ec235660663eceb1c7          13
5fb141ba6677855a1ef1f7a5           1
5fc6c66db312473beedf9d4a           1
5fcbc41d70359f1c4435765a           4


In [93]:
############## Preprocessing everything done for data_fb_unbalanced ##############

pid_subjects = df_unbalanced['pid'].unique()
nr_of_trials_per_block = [16,11,11]

for pid in pid_subjects:
    c3 = df_unbalanced['pid']==pid
    blocks = df_unbalanced[c3]['block'].max()
    for b in range(blocks):
        trial= nr_of_trials_per_block[b]
        c2 = (df_unbalanced['trial']==trial)
        c1 = (df_unbalanced['trial']==trial+1)
        c4 = (df_unbalanced['block']==b+1)
        update = df_unbalanced[c1 & c3 & c4]['confidence'].values[0]-df_unbalanced[c2 & c3 &c4]['confidence'].values[0]
        avg_baseline= df_unbalanced[c3 & c4 & (df_unbalanced['trial'].isin(range(trial+1)))]['confidence'].mean()
        avg_valence= df_unbalanced[c3 & c4 & (df_unbalanced['trial'].isin(range(trial+1, trial+20)))]['confidence'].mean()
        avg_update = avg_valence - avg_baseline
        df_unbalanced.loc[c3&c4, 'single_update'] = update
        df_unbalanced.loc[c3&c4, 'avg_update'] = avg_update
 
        perf_update = df_unbalanced[c1 & c3 & c4]['pavg'].values[0]-df_unbalanced[c2 & c3 &c4]['pavg'].values[0] 
        avg_perf_baseline= df_unbalanced[c3 & c4 & (df_unbalanced['trial'].isin(range(trial+1)))]['pavg'].mean()
        avg_perf_valence= df_unbalanced[c3 & c4 & (df_unbalanced['trial'].isin(range(trial+1, trial+20)))]['pavg'].mean()#
        avg_perf_update = avg_perf_valence - avg_perf_baseline
        df_unbalanced.loc[c3&c4, 'perf_single_update'] = perf_update
        df_unbalanced.loc[c3&c4, 'perf_avg_update'] = avg_perf_update
        
df_unbalanced.to_csv('data_fb_unbalanced.csv', index=False)
df = df_unbalanced[df_unbalanced['pid'].isin(df_balanced['pid'].unique())]
df.to_csv('data_fb_balanced.csv', index=False)


#trials
for pid in pid_subjects:
    c3 = df['pid']==pid
    for b, trial in enumerate([16,11,11]):   
        c2 = (df['trial']==17)
        c4 = (df['block']==b+1)
        df.loc[c3&c4, 'condition_fb'] = df[c2&c4&c3]['condition'].values[0]
        
        
df = pd.read_csv('data_all_blocks.csv')


df_trials = df_trials.drop(columns=['diff', 'diff_abs', 'diff_log', 'rt'])
df_means = df_full.groupby(['block', 'pid', 'trial'])[['diff', 'diff_abs', 'diff_log', 'rt']].mean().reset_index()
df_trials = df_trials.merge(df_means, on=['block', 'pid', 'trial'], how='left')
for i in ['diff','diff_abs', 'diff_log', 'rt']:
    df_trials = df_trials.rename(columns = {f'{i}': f'{i}_mean'})

df_trials['feedback_shifted_1'] = df_trials.groupby(['pid'])['feedback'].apply(lambda x: x.shift(1))
df_trials['condition_shifted_1'] = df_trials.groupby(['pid'])['condition'].apply(lambda x: x.shift(1))


# remove subjects 



In [99]:
# WORKAROUND 
# add variable for baseline 
df_trials = pd.read_csv('/Users/annelene/Documents/uni/phd/metacognition/code/analysis/analysis-V3/analysis-V3-main/data_fb_unbalanced.csv')
df_trials['baseline']=0
df_trials.loc[df_trials['condition']=='baseline', 'baseline']=1
df_trials = df_trials[~df_trials['pid'].isin(['5f10da001b822c2988fbdb89', '5fbe9e4810850d05920f1d58', '5fbd4036ab05790a26289296'])]


In [100]:
# Store dataframes
df_trials.to_csv(f'./../data/processed/fixed_feedback_data_processed_trials.csv')
df_full.to_csv(f'./../data/processed/fixed_feedback_data_processed_subtrials.csv')
